In [1]:
# Import nilearn and numpy
from nilearn import datasets
from nilearn import surface
from nilearn import plotting
import numpy as np
import pandas as pd
import glob
from nilearn import signal
from scipy import stats
import nibabel as nib
%matplotlib inline


In [ ]:
# CHECK FOR MISSING DATA
subs = ['107', '108', '173', '230', '309', '984', '1210', '1211', '1215','1234', '1335', '1339','1380','1452','1453']

for ii in range(len(subs)):
    datadir = ('/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-' + subs[ii] + '/ses-2/func/')
    run1gii = (datadir + 'sub-' + subs[ii] + '_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii')
    print(glob.glob(run1gii))
    run2gii = (datadir + 'sub-' + subs[ii] + '_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii')
    print(glob.glob(run2gii))

['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-107/ses-2/func/sub-107_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-107/ses-2/func/sub-107_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-108/ses-2/func/sub-108_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-108/ses-2/func/sub-108_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-173/ses-2/func/sub-173_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-173/ses-2/func/sub-173_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii']
['/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-230/ses-2/func/sub-230_ses-2_task-rest_run-1_space-fsaverage_hemi-L_b

In [ ]:
# Define ROIs
# pOTS-words, pFus-Faces, OTS-Bodies
roi_names = ['JDY_IFS_HandDrawn','JDY_IPS_HandDrawn','JDY_TOS_HandDrawn','MPM_lh_CoS',
    'MPM_lh_hMT','MPM_lh_IOG','MPM_lh_IOS_IOG','MPM_lh_IOS','MPM_lh_ITG','MPM_lh_LOS',
    'MPM_lh_mFus','MPM_lh_MTG','MPM_lh_OTS','MPM_lh_pFus','MPM_lh_pOTS','MPM_lh_v1d',
    'MPM_lh_v1v','MPM_lh_v2d','MPM_lh_v2v','MPM_lh_v3d','MPM_lh_v3v']
# Define subjects
subs = ['107', '108', '173', '230', '309', '984', '1210','1234', '1335', '1339','1380','1452','1453']
# Missing runs  '1211' , '1215'

# Directory to save output data
outdir = '/scratch/groups/jyeatman/LMB_Connectivity/freesurfer/'
imagedir = '/scratch/groups/jyeatman/LMB_Connectivity/images/'
# Load fsaverage
fsaverage = datasets.fetch_surf_fsaverage('fsaverage')

# Allocate empty array for group data
group_map = np.zeros(shape = (163842,len(subs)))

# Loop ROIs and subjects
for rr in range(len(roi_names)):
    # Load first ROI
    vwfa1 = surface.load_surf_data('/scratch/groups/jyeatman/visfAtlas/FreeSurfer/' + roi_names[rr] + '.label')
    print('Analyzing ROI /scratch/groups/jyeatman/visfAtlas/FreeSurfer/' + roi_names[rr] + '.label')
    
    # Loop over subjects and compute connectivity for that ROI
    for ii in range(len(subs)):
        datadir = ('/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-' + subs[ii] + '/ses-2/func/')
        run1gii = (datadir + 'sub-' + subs[ii] + '_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii')
        print('Loading data for participant ' + subs[ii])
        print('Loading run 1:' + run1gii)
        run1 = surface.load_surf_data(run1gii)
        run2gii = (datadir + 'sub-' + subs[ii] + '_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii')
        print('Loading run 1:' + run2gii)
        run2 = surface.load_surf_data(run2gii)

        # Loading confounds
        con1 = pd.read_csv(datadir + 'sub-' + subs[ii] + '_ses-2_task-rest_run-1_desc-confounds_timeseries.tsv', sep = '\t')
        con2 = pd.read_csv(datadir + 'sub-' + subs[ii] +  '_ses-2_task-rest_run-2_desc-confounds_timeseries.tsv', sep = '\t')
        con1 = con1[['csf', 'white_matter','trans_x', 'trans_x_derivative1', 'trans_y','trans_y_derivative1','trans_z', 
                 'trans_z_derivative1', 'rot_x', 'rot_x_derivative1','rot_y','rot_y_derivative1', 
                 'rot_z', 'rot_z_derivative1']]
        con2 = con2[['csf', 'white_matter','trans_x', 'trans_x_derivative1', 'trans_y','trans_y_derivative1','trans_z', 
                 'trans_z_derivative1', 'rot_x', 'rot_x_derivative1','rot_y','rot_y_derivative1', 
                 'rot_z', 'rot_z_derivative1']]

        # drop first 6 timepoints
        droptp = [0,1,2,3,4,5]
        print('Original Data Vertices by Timepoints')
        print(run1.shape)
        run1 = np.delete(run1,droptp,axis=1)
        run2 = np.delete(run2,droptp,axis=1)
        con1 = con1.drop(droptp)
        con2 = con2.drop(droptp)
        print('Vertices by Timepoints after dropping initial 6 timepoints')
        print(run1.shape)
        print(run2.shape)
        print(con1.shape)
        print(con2.shape)

        # Signal clean
        run1 = np.transpose(signal.clean(signals = np.transpose(run1), filter=None, detrend=True, standardize='zscore',confounds = con1))
        run2 = np.transpose(signal.clean(signals = np.transpose(run2), filter=None, detrend=True, standardize='zscore',confounds = con2))
        runs = np.concatenate((run1,run2),axis=1)
        print(runs.shape)

        # Compute the mean time series for the ROI
        seed_timeseries = np.mean(runs[vwfa1], axis=0)

        # Plot mean timeseries
        #import matplotlib.pyplot as plt

        #plt.plot(seed_timeseries)
        #plt.title('Seed time series (Posterior cingulate cortex)')
        #plt.xlabel('Scan number')
        #plt.ylabel('Normalized signal')
        #plt.tight_layout()

        # Compute correlations between the seed timeseries and each vertex
        stat_map = np.zeros(run1.shape[0])
        for i in range(run1.shape[0]):
            stat_map[i] = stats.pearsonr(seed_timeseries, runs[i])[0]

        # Save as a gifti that could be loaded into freeview
        nib.freesurfer.io.write_morph_data(outdir + subs[ii] + '_connectivity_' + roi_names[rr] + '.curv',stat_map)

        # Add the stat map to the group stat map
        group_map[:,ii] = stat_map

        # Now plot the seed based connectivity
        plotting.plot_surf_stat_map(fsaverage['white_left'], stat_map=stat_map,
            hemi='left', threshold = .3, vmax=0.7, view='lateral', colorbar=True,
            bg_map=fsaverage['curv_left'], title='Correlation map', output_file = (imagedir + 'sub-' + subs[ii] + '_ses-2_' + roi_names[rr] + '_lateral.png' ))

        plotting.plot_surf_stat_map(fsaverage['white_left'], stat_map=stat_map,
            hemi='left', threshold = .3, vmax=0.7, view='ventral', colorbar=True,
            bg_map=fsaverage['curv_left'], title='Correlation map', output_file = (imagedir + 'sub-' + subs[ii] + '_ses-2_' + roi_names[rr] + '_ventral.png' ))

        
    # Calculate the mean for roi_names[rr]
    print('Calculating mean connectivity for' + roi_names[rr])
    group_mean = np.mean(group_map, axis = 1)
    nib.freesurfer.io.write_morph_data(outdir + 'GroupMean_connectivity_' + roi_names[rr] + '.curv',stat_map)



Analyzing ROI /scratch/groups/jyeatman/visfAtlas/FreeSurfer/JDY_IFS_HandDrawn.label
Loading data for participant 107
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-107/ses-2/func/sub-107_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-107/ses-2/func/sub-107_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)


/home/users/jyeatman/.local/lib/python3.6/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Loading data for participant 108
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-108/ses-2/func/sub-108_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-108/ses-2/func/sub-108_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)
Loading data for participant 173
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-173/ses-2/func/sub-173_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-173/ses-2/func/sub-173_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369

Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)
Loading data for participant 309
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-309/ses-2/func/sub-309_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-309/ses-2/func/sub-309_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)
Loading data for participant 984
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-984/ses-2/func/sub-984_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-984/ses-2/func/sub-984_ses-2_task-rest_run-2_space-f

Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-1210/ses-2/func/sub-1210_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)
Loading data for participant 1234
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-1234/ses-2/func/sub-1234_ses-2_task-rest_run-1_space-fsaverage_hemi-L_bold.func.gii
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-1234/ses-2/func/sub-1234_ses-2_task-rest_run-2_space-fsaverage_hemi-L_bold.func.gii
Original Data Vertices by Timepoints
(163842, 375)
Vertices by Timepoints after dropping initial 6 timepoints
(163842, 369)
(163842, 369)
(369, 14)
(369, 14)
(163842, 738)
Loading data for participant 1335
Loading run 1:/scratch/groups/jyeatman/LMB_BIDS/derivatives/fmriprep/sub-1335/ses-2/func/sub-1335_ses-2_task-rest_run

In [ ]:
# Now plot the seed based connectivity
plotting.plot_surf_stat_map(fsaverage['white_left'], stat_map=group_mean,
        hemi='left', threshold = .2, vmax=0.5, view='lateral', colorbar=True,
        bg_map=fsaverage['curv_left'], title='Correlation map', output_file = ('/scratch/groups/jyeatman/images/' + 'Group_Mean_Lateral' + '_ses-2_MPM_lh_OTS.png' ))
    
# Now plot the seed based connectivity
plotting.plot_surf_stat_map(fsaverage['white_left'], stat_map=group_mean,
        hemi='left', threshold = .2, vmax=0.5, view='ventral', colorbar=True,
        bg_map=fsaverage['curv_left'], title='Correlation map', output_file = ('/scratch/groups/jyeatman/images/' + 'Group_Mean_Lateral' + '_ses-2_MPM_lh_OTS.png' ))
    
    

In [ ]:
# Transform ROI indices to a "roi_map" which is what nilearn calls a mask
# Plot this ROI on the cortical surface
vwfa1_map = np.zeros(run1.shape[0], dtype=int)
vwfa1_map[vwfa1] = 1
from nilearn import plotting
plotting.plot_surf_roi(fsaverage['white_left'], roi_map=vwfa1_map,
                       hemi='left', view='ventral',
                       bg_map=fsaverage['curv_left'], bg_on_data=True,
                       title='VWFA1 Seed')

In [ ]:
# Plot on ventral surface
plotting.plot_surf_stat_map(fsaverage['white_left'], stat_map=stat_map,
                            hemi='left', threshold = .3,vmax = 0.7, view='ventral', colorbar=True,
                            bg_map=fsaverage['curv_left'], title='Correlation map')